<a href="https://colab.research.google.com/github/TheDumbEngineer/AI_Assistants/blob/main/ReverseEngineeringAssistantV2_0_RAG_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Reverse Engineering Assistant V2.0


In [11]:
!pip install langchain langchain_community langchain_chroma langchain-openai langchainhub unstructured[pdf]

ERROR: Could not find a version that satisfies the requirement langchain.runnables (from versions: none)
ERROR: No matching distribution found for langchain.runnables


In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

··········


In [2]:
#Using the UnstructuredPDFLoader for loading the document
from langchain_community.document_loaders import UnstructuredPDFLoader

In [3]:
#Create the loader
loader = UnstructuredPDFLoader("/content/drive/MyDrive/Book PDFs/PracticalBinary.pdf")

In [4]:
#Get the loaded data
data = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [9]:
#Importing the libraries
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain import hub
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chains import TransformChain
from langchain.chat_models import ChatOpenAI
import sys

ImportError: cannot import name 'RunnableMap' from 'langchain.runnables' (/usr/local/lib/python3.10/dist-packages/langchain/runnables/__init__.py)

In [6]:
#Splitting the data and creating a vectorestore
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(data)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [44]:
#Retrieving and generating using the relevant data from the pdf.
retriever = vectorstore.as_retriever()
prompt = hub.pull("byteberzerker/reverse_helper")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("Give me a step by step guide on binary analysis, complete with the tools used and how to use them")





{'question': 'Hello', 'chat_history': [HumanMessage(content='Hello'), AIMessage(content='Hello! It seems like the context provided is incomplete and does not directly relate to the question asked. If you could provide more information or clarify your question, I would be happy to assist you further.')], 'answer': 'Hello! It seems like the context provided is incomplete and does not directly relate to the question asked. If you could provide more information or clarify your question, I would be happy to assist you further.'}


#With Nice Formatting, No Memory

In [ ]:
retriever = vectorstore.as_retriever()
prompt = hub.pull("byteberzerker/reverse_helper")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

result = rag_chain.invoke("Give me some SQL examples used in fuzzing APIs.")


# Function to print the result nicely
def print_nice_result(result):
    print("\n=== RAG Chain Result ===\n")
    sections = result.split("\n\n")
    for idx, section in enumerate(sections, start=1):
        print(f"Section {idx}:\n{section.strip()}\n")
        print("=" * 40)

# Print the nicely formatted result
print_nice_result(result)


=== RAG Chain Result ===

Section 1:
To perform SQL fuzzing on APIs, you need to follow these steps:

Section 2:
1. Identify the API endpoints that you want to fuzz. Look for inputs that are used for user interaction, such as authentication forms, account registration, file uploads, content editing, user profile editing, user management, and content searching.

Section 3:
2. Choose the type of input you want to send for fuzzing. This can include symbols, strings, and numbers that could potentially cause errors in the API.

Section 4:
3. Craft SQL fuzzing payloads that target the identified API endpoints. For example, you can send SQL injection payloads to input fields that interact with a database.

Section 5:
4. Send the SQL fuzzing payloads to the API endpoints and observe the responses. Look for verbose error messages or other indications that the API is vulnerable to SQL injection attacks.

Section 6:
5. Adapt your fuzzing based on the results. If you receive error messages indica